# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [9]:
#Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [3]:
#Your code here
cur.execute("""SELECT lastName, firstName FROM employees
               JOIN offices
               ON employees.officeCode = offices.officeCode
               WHERE city = 'Boston';""").fetchall()

[('Firrelli', 'Julie'), ('Patterson', 'Steve')]

## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [5]:
#Your code here
cur.execute("""SELECT city, COUNT(lastName)
            FROM offices
            JOIN employees
            USING(officeCode)
            GROUP BY city;""").fetchall()

[('Boston', 2),
 ('London', 2),
 ('NYC', 2),
 ('Paris', 5),
 ('San Francisco', 6),
 ('Sydney', 4),
 ('Tokyo', 2)]

## Write 3 Questions of your own and answer them

In [17]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
cur.execute("""SELECT htmlDescription FROM productlines""").fetchall()

[('',), ('',), ('',), ('',), ('',), ('',), ('',)]

Display the productLine and employee's first and last name for each product that each employee has sold

In [66]:
# Your code here
cur.execute("""SELECT productLine, lastName, firstName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               JOIN products
               USING(productCode)
               JOIN productlines
               USING(productLine);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df

2996


,productLine,lastName,firstName
0,Trucks and Buses,Jennings,Leslie
1,Trucks and Buses,Jennings,Leslie
2,Vintage Cars,Jennings,Leslie
3,Trucks and Buses,Jennings,Leslie
4,Classic Cars,Jennings,Leslie
...,...,...,...
2991,Trucks and Buses,Gerard,Martin
2992,Trains,Gerard,Martin
2993,Trucks and Buses,Gerard,Martin
2994,Vintage Cars,Gerard,Martin


Display top 5 sales employees

In [31]:
# Your code here
cur.execute("""SELECT lastName, firstName, sum(priceEach) AS total_sales
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               GROUP BY employeeNumber
               ORDER BY total_sales DESC
               LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,lastName,firstName,total_sales
0,Hernandez,Gerard,34668.30
1,Jennings,Leslie,30218.96
2,Castillo,Pamela,25148.59
3,Bott,Larry,20898.67
4,Jones,Barry,20700.90


Sales rep by office

In [91]:
cur.execute("""SELECT city, jobTitle, lastName, firstName
               FROM offices
               LEFT JOIN employees
               USING(officeCode)
               WHERE jobTitle = 'Sales Rep'
               ORDER BY city ASC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,city,jobTitle,lastName,firstName
0,Boston,Sales Rep,Firrelli,Julie
1,Boston,Sales Rep,Patterson,Steve
2,London,Sales Rep,Bott,Larry
3,London,Sales Rep,Jones,Barry
4,NYC,Sales Rep,Tseng,Foon Yue
5,NYC,Sales Rep,Vanauf,George
6,Paris,Sales Rep,Bondur,Loui
7,Paris,Sales Rep,Hernandez,Gerard
8,Paris,Sales Rep,Castillo,Pamela
9,Paris,Sales Rep,Gerard,Martin


## Level Up: Display the names of every individual product that each employee has sold

In [89]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders 
               USING(customerNumber)
               JOIN orderdetails 
               USING(orderNumber)
               JOIN products 
               USING(productCode);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up: Display the Number of Products each employee has sold

In [86]:
# Your code here
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               FROM employees e
               JOIN customers c                   
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o 
               USING(customerNumber)
               JOIN orderdetails od 
               USING(orderNumber)                    
               JOIN products p 
               USING(productCode)
               GROUP BY employeeNumber;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

15


,firstName,lastName,COUNT(productName)
0,Leslie,Jennings,331
1,Leslie,Thompson,114
2,Julie,Firrelli,124
3,Steve,Patterson,152
4,Foon Yue,Tseng,142
5,George,Vanauf,211
6,Loui,Bondur,177
7,Gerard,Hernandez,396
8,Pamela,Castillo,272
9,Larry,Bott,236


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!